# Pretrained Model Loading 

In [ ]:
import torch 
from torchvision import models 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg = models.vgg16(pretrained=True).to(device)
print(vgg)

In [ ]:
from torchsummary import summary 
summary(vgg, (3, 224, 224))

In [ ]:
for name, layer in vgg.named_modules():
    print(name, layer)

In [ ]:
vgg.fc = torch.nn.Linear(1000, 1)
vgg.cpu()

In [ ]:
vgg = models.vgg16(pretrained=True).to(device)
print(vgg)

In [ ]:
vgg.classifier._modules['6'] = torch.nn.Linear(4096, 1)
vgg.cpu()

# Pretrained Model

In [1]:
from torchvision import datasets
import torchvision.transforms as transforms 
import torch 

dataset = datasets.ImageFolder(root='./data/',
    transform=transforms.Compose([
        transforms.Resize(size=(244,244)),
        #transforms.Scale(244),
        transforms.CenterCrop(244),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=8)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from torch import nn 
from torchvision import models 

class MyNewNet(nn.Module):
    def __init__(self):
        super(MyNewNet, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        self.linear_layer = nn.Linear(1000,1)
    
    def forward(self, x):
        x = self.vgg19(x)
        return self.linear_layer(x)

In [7]:
def create_MyNewNet():
    my_model = MyNewNet()
    my_model = my_model.to(device)

    for param in my_model.parameters():
        param.requires_grad = False
    
    for param in my_model.linear_layer.parameters():
        param.requires_grad = True 
    
    return my_model 

my_model = create_MyNewNet()

In [8]:
# print(my_model)
# print(my_model.eval())

# x = next(iter(dataloader))
# x[0].shape

# my_model(x[0].to(device))
# x[1]

In [15]:
EPOCHS = 10#100
BATCH_SIZE = 64
LEARNING_RATE = 0.0005

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(100*acc)

    return acc 

In [16]:
next(my_model.vgg19.features._modules['0'].parameters())[0]

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]])

In [17]:
it = my_model.linear_layer.parameters()
next(it)[0][:10]

tensor([-0.0019,  0.0006, -0.0070, -0.0179,  0.0324,  0.0031,  0.0096, -0.0038,
         0.0138,  0.0022], grad_fn=<SliceBackward0>)

In [18]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    num_workers=8
)

#my_model = create_MyNewNet()

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(my_model.parameters(), lr=LEARNING_RATE)

for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0

    for X_batch, y_batch in dataloader:
        if device.type == 'cpu':
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.FloatTensor)
        else:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.FloatTensor).type(torch.cuda.FloatTensor)

        optimizer.zero_grad()
        y_pred = my_model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    print(f'Epoch {e+0:03}: | loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

Epoch 001: | loss: 1.90003 | Acc: 54.571
Epoch 002: | loss: 0.90107 | Acc: 69.429
Epoch 003: | loss: 0.68966 | Acc: 65.571
Epoch 004: | loss: 0.88890 | Acc: 57.857
Epoch 005: | loss: 0.81608 | Acc: 58.286
Epoch 006: | loss: 0.78900 | Acc: 56.714
Epoch 007: | loss: 0.69064 | Acc: 61.000
Epoch 008: | loss: 0.64063 | Acc: 63.429
Epoch 009: | loss: 0.68716 | Acc: 59.429
Epoch 010: | loss: 0.68693 | Acc: 58.429


In [19]:
next(my_model.vgg19.features._modules['0'].parameters())[0]

tensor([[[-0.0535, -0.0493, -0.0679],
         [ 0.0153,  0.0451,  0.0021],
         [ 0.0362,  0.0200,  0.0199]],

        [[ 0.0170,  0.0554, -0.0062],
         [ 0.1416,  0.2271,  0.1376],
         [ 0.1200,  0.2003,  0.0921]],

        [[-0.0449,  0.0127, -0.0145],
         [ 0.0597,  0.1395,  0.0541],
         [-0.0010,  0.0583, -0.0297]]])

In [20]:
it = my_model.linear_layer.parameters()
next(it)[0][:10]

tensor([-0.0068, -0.0018, -0.0077, -0.0202,  0.0281,  0.0066,  0.0067, -0.0067,
         0.0100, -0.0031], grad_fn=<SliceBackward0>)